In [ ]:
# 可能要做的exception handling
#   1. drop column 時可能會對column 名稱做更改，還不知道怎麼處理b

# 身障身要需要身障房才會被安排到

In [1]:
import sys
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from random import *
from qua_config import *


In [2]:
# 如果沒志願，就加一般區
# column 都變成 config


# distinguish type of identities
# the list that push to front
def ori_id_set(origin_df):
    id_col_name  = ['身分別1','身分別2','身分別3','特殊身份別']
    column       = [str(x) for i in range(len(id_col_name)) for x in origin_df[id_col_name[i]].tolist() if str(x)!='nan']
    return list(set(column))


# show the list to the front and get the arranged id_list order back
# assume we have the ordered list
def get_id_dict(id_list):
    index_value   = 1
    id_dict       = dict()
    for i in range(len(id_list)):
        if(id_list[i] not in id_dict.keys()):
            id_dict[id_list[i]]   =  index_value
            index_value           += 1
    return id_dict


# organized origin df
# didn't consider if identity need to be qualified

# 身心障礙、低收入戶、中低收入戶、離島、原住民、僑外、國際生、
# 外交人員子女、醫學系公費生、北北基及桃園以外的縣市、桃園

# exception: 
#    1. 身障生、中低收入戶、離島生、自選身份 四個要審核, 寫死了
#    2. 
def get_str2int(origin_df):
    id_list               = ori_id_set(origin_df)
    # sent id_list to front
    # receive order list from front
#     id_dict               = get_id_dict(id_list)

    # temp use default id_dict
    
    id_index                           = [0 for i in range(len(origin_df))]
    willing_index                      = [0 for i in range(len(origin_df))]
    for index, row in origin_df.iterrows():
        # check if have special identity_type & audit status
        for i in id_col_name:
            if(str(row[i])!='nan' and row[i] in id_dict.keys() and id_dict[row[i]]>id_index[index]):
                if(i == '特殊身份別' or row[i] in audit_list):
                    if(row['審查狀態']=='通過'):
                        id_index[index]    = id_dict[row[i]]
                else:
                    id_index[index]    = id_dict[row[i]]
        # no special identity_type, check the habitation
        if(id_index[index] == 0):
            id_index[index]        = len(id_dict) + 2 if row['戶籍地'] in last_order_habitation else len(id_dict)+1
        # willing_index assign
        if(str(row['校內外意願'])!='nan'):
            willing_index[index]   = willing_InOutCam[row['校內外意願']]
    origin_df['校內外意願']          = willing_index
    origin_df['id_index']          = id_index
    return origin_df


# select qualification function

# input the df that order by id_index
def assign_qualificaiton(df,bedNum =100):
    id_count                = df.groupby('id_index')['id_index'].count().tolist()
    curI                    = 0
    if(df.at[0,'id_index'] == 1):
        curI                = id_count[0]
        id_count            = id_count[1:]
    while(bedNum>0):
        if(len(id_count)>0):
            RandomOrder     = sample([i for i in range(id_count[0])],k=id_count[0]) if(id_count[0]<bedNum) else sample([i for i in range(id_count[0])],k=bedNum)
            for i in RandomOrder:
                df.at[curI+i,'資格'] = 1
                bedNum          -= 1
                if(bedNum ==0 ):
                    break
            curI            = curI + id_count[0]
            id_count        = id_count[1:] 
        else:
            break
    return df

# alOutCam

def alCamOrderAssign(df):
    id_count            = df.groupby('id_index')['id_index'].count().tolist()
    curI                = 0
    OnListNum           = 1
    for idC in id_count:
        RandomOrder     = sample([i for i in range(idC)],k=idC)
        for i in RandomOrder:
            df.at[curI+i,'資格'] = OnListNum
            OnListNum           += 1
        curI            = curI + idC
    return df
            

def MainFunc(origin_df):
    # execute
    # original data with deleted no needed column, id_type => int
    origin_df      = origin_df.drop(columns=Ori_ColumnToBeDrop)
    origin_df       = get_str2int(origin_df)

    
    # get inCampus df & alOutCam
    # organize df
    origin_df          = origin_df.sort_values(by = '校內外意願').reset_index(drop = True)
#     InCamNum           = len([i for i in origin_df['校內外意願'] if i != 3])
    InCamNum           = len(origin_df)-origin_df.groupby(['校內外意願']).size()[2]
    
    # inCam_df

    InCam_df           = origin_df.iloc[:InCamNum,:]
    InCam_df           = InCam_df.sort_values(by = '性別').drop(columns=InCam_ColumnToBeDrop).reset_index(drop = True)


    InCam_df['宿舍']    = 0
    InCam_df['房號']    = 0
    InCam_df['床位']    = 0
    InCam_df['資格']    = [2 if (row['id_index']==1) else 0 for index,row in InCam_df.iterrows()]
    GirlInCamNum       = InCam_df.groupby(['性別']).size()['女性']
    GirlInCam          = InCam_df.iloc[:GirlInCamNum,:].sort_values(by='id_index').reset_index(drop=True)
    BoyInCam           = InCam_df.iloc[GirlInCamNum:,:].sort_values(by='id_index').reset_index(drop=True)

    OutCam_Aldf        = origin_df.iloc[InCamNum:,:]


    # get qualification of boy&girl df
    GirlInCam  = assign_qualificaiton(GirlInCam,BedNum)
    BoyInCam  = assign_qualificaiton(BoyInCam,BedNum)
    GirlInCam          = GirlInCam.sort_values(by='資格').reset_index(drop=True)
    BoyInCam           = BoyInCam.sort_values(by='資格').reset_index(drop=True)


    # InCam_alDf
    alGirlNum          = GirlInCam.groupby('資格')['資格'].count().tolist()[0]
    alBoyNum           = BoyInCam.groupby('資格')['資格'].count().tolist()[0]
    twoAlDf            = [GirlInCam.iloc[:alGirlNum,:],BoyInCam.iloc[:alBoyNum]]
    InCam_Aldf         = pd.concat(twoAlDf)

    # Output Girl&Boy df
    GirlInCam          = GirlInCam.iloc[alGirlNum:,:]
    BoyInCam           = BoyInCam.iloc[alBoyNum:,:]


    # organize Alternative df
    InCam_Aldf         = InCam_Aldf.drop(columns=InCamAl_ColumnToBeDrop)


    OutCam_Aldf        = OutCam_Aldf.drop(columns=OutCamAl_ColumnToBeDrop)



    # add willingness=1 to OutCam
    InCam_Aldf         = InCam_Aldf.sort_values(by='校內外意願',ascending=False).reset_index(drop=True)
    willingness2Num    = InCam_Aldf.groupby('校內外意願')['校內外意願'].count().tolist()[1]
    will2_df           = InCam_Aldf.iloc[:willingness2Num,:]
    OutCam_Aldf        = OutCam_Aldf.append(will2_df)


    InCam_Aldf         = InCam_Aldf.sort_values(by='id_index').reset_index(drop=True)
    OutCam_Aldf        = OutCam_Aldf.sort_values(by='id_index').reset_index(drop=True)



    # alternative order giving
    OutCam_Aldf = alCamOrderAssign(OutCam_Aldf)
    
    
    # has to combine with 
    InCam_Aldf  = alCamOrderAssign(InCam_Aldf)
    
    

    return GirlInCam, BoyInCam, InCam_Aldf, OutCam_Aldf

In [8]:
# no need "是否需要安排身障房"
#test param
BedNum = 100
origin_df      = pd.read_csv('BallotApplyUndergraduate.csv', header = 0)



# GirlInCam, BoyInCam, InCam_Aldf, OutCam_Aldf = MainFunc(origin_df)



In [9]:
origin_df

,系所代碼,系所名稱,年級,學號,姓名,性別,入學年度,身分別,身分別1,身分別2,...,郵遞區號,通訊地址,聯絡人姓名,與聯絡人關係,聯絡人電話,聯絡人手機,送出時間,現住床位,最後修改,操作人員
0,4090,職能治療學系,1,B08***027,何OO,女性,108,83,僑生,分發生,...,853.0,澳門新口岸高美***************************,陳OO,外袓母,02-2*****16,0976*****2,2019-08-19 16:14,NaN,2019-08-19 17:16,馮安華
1,7020,會計學系,1,B08***033,何OO,男性,108,14,本國生,個人申請入學生,...,407.0,臺中市西屯區寶***************************,黃OO,母子,04-2*****49,0975*****5,2019-08-19 13:57,NaN,NaN,NaN
2,6050,森林環境暨資源學系,1,B08***040,白OO,男性,108,1Q,本國生,NaN,...,221.0,新北市汐止區興***************************,白OO,父子,02-2*****22,0965*****6,2019-08-19 13:51,NaN,2019-08-19 15:49,馮安華
3,1020,外國語文學系,1,B08***126,曾OO,男性,108,11,本國生,考試分發入學,...,807.0,高雄市三民區山***************************,曾OO,父子,06-2*****7,*****,2019-08-19 12:54,NaN,NaN,NaN
4,7030,財務金融學系,1,B08***130,鄭OO,男性,108,11,本國生,考試分發入學,...,542.0,南投縣草屯鎮文***************************,錢OO,母←,049-*****52,*****,2019-08-19 12:51,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999,5040,化學工程學系,1,B08***087,麥OO,女性,108,5,外籍生,NaN,...,106.0,臺北市大安區羅***************************,OO,NaN,*****,*****,NaN,NaN,NaN,NaN
2000,5040,化學工程學系,1,B08***085,文OO,男性,108,5,外籍生,NaN,...,NaN,Simei S***************************ngapore,OO,NaN,*****,*****,NaN,NaN,NaN,NaN
2001,2090,大氣科學系,1,B08***027,李OO,女性,108,5,外籍生,NaN,...,NaN,"706dong***************************163, Byeorya...",OO,NaN,*****,*****,NaN,NaN,NaN,NaN
2002,3030,經濟學系,1,B08***069,齊OO,男性,108,5 I,外籍生,NaN,...,NaN,日本國東京都中***************************,OO,NaN,*****,*****,NaN,NaN,NaN,NaN


In [5]:
GirlInCam

,學號,性別,校內外意願,區域志願1,區域志願2,區域志願3,永久地址,是否需要安排身障房間,id_index,宿舍,房號,床位,資格
459,B08***043,女性,1,國際互動區,一般區域,健康作息區,臺北市北投區奇******************************,否,2,0,0,0,1
460,B08***020,女性,1,運動休閒區,國際互動區,一般區域,宜蘭縣蘇澳鎮永******************************,否,2,0,0,0,1
461,B08***038,女性,1,整潔模範區,健康作息區,國際互動區,臺南市永康區正******************************,否,2,0,0,0,1
462,B08***021,女性,1,一般區域,國際互動區,健康作息區,台南市東區關聖******************************,是,1,0,0,0,2


In [50]:
OutCam_Aldf = alCamOrderAssign(OutCam_Aldf)
OutCam_Aldf

,學號,校內外意願,永久地址,id_index,資格
0,B08323432,3,NaN,1,1.0
1,B08394098,2,NaN,8,2.0
2,B08323432,2,NaN,11,3.0
3,B08323432,2,NaN,11,4.0
